In [210]:
# set up the data set
from datetime import time
import pandas as pd

data = pd.read_excel(r"newInput.xlsx")
df = pd.DataFrame(data)

In [211]:
"""defining functions to converts to seconds and then  back to time object """
def convertToMin(time_obj):
    return (time_obj.hour * 60 + time_obj.minute)

def convertToTime(minutes):
    hour, minute = divmod(minutes, 60)
    return time(hour, minute)

In [212]:
# can create a additional function to call this in future development 
strToTime = lambda strTime: time(*[int(x) for x in strTime.split(':')])
# DEFINING GLOBAL VARIABLES 
sorted_set = df.sort_values('Departure Time', ascending=True).reset_index(drop=True)
start_set = sorted_set[sorted_set['Departure Place'] == 'PSL'].reset_index(drop=True)

# GLOBAL VARIABLES
# all the time constraints are mentioned in minutes
minDuty = 8 * 60 
maxDuty = 10 * 60 
# minSpreadOver = 10 * 60 
maxSpreadOver = 12 * 60 
trip = [{"selected_leg":start_set.iloc[0].to_frame().T, 
         "temp_set": start_set.drop(0).reset_index(drop=True),
         "duty": convertToMin(strToTime(start_set.iloc[0]['Running Time'])),
         }]

In [213]:
def getSpreadOver():
    global trip
    start = strToTime(trip[0]['selected_leg']['Departure Time'][0])
    end = strToTime(trip[-1]['selected_leg']['Arrival Time act'][0])
    return (convertToMin(end) - convertToMin(start))

def setDuty(leg):
    global trip 
    duty = trip[-1]['duty']
    duty += convertToMin(strToTime(leg['Running Time'][0]))
    last_arrival = convertToMin(strToTime(trip[-1]['selected_leg']['Arrival Time act'][0]))
    new_departure = convertToMin(strToTime(leg['Departure Time'][0]))
    terminal_gap = new_departure - last_arrival
    if terminal_gap < 15:
        duty += terminal_gap
    return (terminal_gap, duty)


def getCurrentTrip():
    global trip
    route = [x['selected_leg'] for x in trip]
    display(pd.concat(route).reset_index(drop=True))
    print("Steering Duty = ", convertToTime(trip[-1]['duty']))
    print("Spread Over = ", convertToTime(getSpreadOver()))



In [214]:
def pushToTrip():
    global trip
    while trip[-1]['selected_leg']['Arrival Place'][0] != 'PSL':
        last_leg = trip[-1]['selected_leg']
        temp_set = df[(df['Departure Place'] == last_leg['Arrival Place'][0]) & (
            df['Departure Time'] > last_leg['Arrival Time act'][0])]
        temp_set = temp_set.sort_values(
            'Departure Time', ascending=True).reset_index(drop=True)
        new_leg = temp_set.iloc[0].to_frame().T
        terminal_gap, duty = setDuty(new_leg)
        if terminal_gap > 15 or duty > maxDuty:
            # trip.pop()
            #trip is invalid and this path cannot be continued by incrementing the 
            #temp set since the terminal gap exceeds
            return 1  
        # elif terminal_gap < 15 or duty > maxDuty:
        #     #terminal gap is satisfied ,so we can incrementing the temp set to find a valid trip 
        #     return 2
        
        """MUST CHECK LATER """
        """elif terminal_gap > 15 or duty < maxDuty:
            #split 
            #so check whether any split already there 
            #check whether it is more than 30 and the credentials are satisfied to consider it as a break 
            return 3"""
        
        trip[-1]['selected_leg']['Terminal Gap'] = terminal_gap
        trip.append({"selected_leg": new_leg,
                     "temp_set": temp_set.drop(0).reset_index(drop=True), 
                     "duty": duty, }
                    )
    return 0 #trip valid 


status = pushToTrip()
print(status)
getCurrentTrip()
# what happens if the first trip itself isn't valid 

0


,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,1,03:40:00,PSL,NH,KLKV,03:45:00,3.5,00:05:00,5.0
1,124,03:50:00,KLKV,NH,TVM,04:55:00,33.7,01:05:00,10.0
2,313,05:05:00,TVM,NH-UDA,KNVLA,06:05:00,32.0,01:00:00,10.0
3,641,06:15:00,KNVLA,UDA,MC,08:00:00,39.0,01:45:00,10.0
4,336,08:10:00,MC,NH,KLKV,10:00:00,40.0,01:50:00,5.0
5,695,10:05:00,KLKV,PVR-VZM-BYPASS-TVM-NH-KLKV,PSL,13:35:00,81.5,03:30:00,NaN


Steering Duty =  09:55:00
Spread Over =  09:55:00


In [215]:
def incrementTempSet():
        #check the duty an dif it is high correct it or else return False 
        temp = trip.pop()
        if len(temp['temp_set']) > 0:
                terminal_gap, duty = setDuty(temp['temp_set'].iloc[0].to_frame().T)
                trip[-1]['selected_leg']['Terminal Gap'] = terminal_gap
                temp['selected_leg'] = temp['temp_set'].iloc[0].to_frame().T
                temp['temp_set'] = (temp['temp_set'].drop(0)).reset_index(drop=True)
                temp['duty'] = duty
                trip.append(temp)
                return True
        else :
                return False

In [216]:
# for i in range(50):
i = 0 
while len(trip) > 0:
    status = pushToTrip()
    # print(status)
    if status == 0:#trip valid 
        print("Trip ", i)
        getCurrentTrip()
        while not incrementTempSet():
            continue
    elif status == 1:
        trip.pop()
        while not incrementTempSet():
            continue
    # elif status == 2:
    #     while x:=not incrementTempSet():
    #         continue
    #     print('haii')
    pushToTrip()
    i += 1

Trip  0


,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,1,03:40:00,PSL,NH,KLKV,03:45:00,3.5,00:05:00,5.0
1,124,03:50:00,KLKV,NH,TVM,04:55:00,33.7,01:05:00,10.0
2,313,05:05:00,TVM,NH-UDA,KNVLA,06:05:00,32.0,01:00:00,10.0
3,641,06:15:00,KNVLA,UDA,MC,08:00:00,39.0,01:45:00,10.0
4,336,08:10:00,MC,NH,KLKV,10:00:00,40.0,01:50:00,5.0
5,695,10:05:00,KLKV,PVR-VZM-BYPASS-TVM-NH-KLKV,PSL,13:35:00,81.5,03:30:00,NaN


Steering Duty =  09:55:00
Spread Over =  09:55:00
Trip  4


,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,1,03:40:00,PSL,NH,KLKV,03:45:00,3.5,00:05:00,5.0
1,124,03:50:00,KLKV,NH,TVM,04:55:00,33.7,01:05:00,10.0
2,313,05:05:00,TVM,NH-UDA,KNVLA,06:05:00,32.0,01:00:00,10.0
3,641,06:15:00,KNVLA,UDA,MC,08:00:00,39.0,01:45:00,10.0
4,341,08:10:00,MC,PTM,NTA,09:10:00,27.7,01:00:00,5.0
5,199,09:15:00,NTA,PTM,MC,10:25:00,27.7,01:10:00,15.0
6,386,10:40:00,MC,NH,KLKV,12:30:00,40.0,01:50:00,5.0
7,72,12:35:00,KLKV,NH,PSL,12:45:00,3.5,00:10:00,NaN


Steering Duty =  08:50:00
Spread Over =  09:05:00
Trip  5


...

,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,1,03:40:00,PSL,NH,KLKV,03:45:00,3.5,00:05:00,5.0
1,124,03:50:00,KLKV,NH,TVM,04:55:00,33.7,01:05:00,10.0
2,313,05:05:00,TVM,NH-UDA,KNVLA,06:05:00,32.0,01:00:00,10.0
3,641,06:15:00,KNVLA,UDA,MC,08:00:00,39.0,01:45:00,15.0
4,343,08:15:00,MC,NH,KLKV,09:50:00,40.0,01:35:00,25.0
5,517,10:15:00,KLKV,KRKM,VLRD,10:55:00,17.0,00:40:00,565.0
6,582,20:20:00,VLRD,KRKM,KLKV,21:00:00,17.0,00:40:00,130.0
7,106,23:10:00,KLKV,NH,PSL,23:15:00,3.5,00:05:00,NaN


Steering Duty =  07:20:00
Spread Over =  19:35:00
Trip  376


,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time,Terminal Gap
0,1,03:40:00,PSL,NH,KLKV,03:45:00,3.5,00:05:00,5.0
1,124,03:50:00,KLKV,NH,TVM,04:55:00,33.7,01:05:00,10.0
2,313,05:05:00,TVM,NH-UDA,KNVLA,06:05:00,32.0,01:00:00,10.0
3,641,06:15:00,KNVLA,UDA,MC,08:00:00,39.0,01:45:00,15.0
4,343,08:15:00,MC,NH,KLKV,09:50:00,40.0,01:35:00,25.0
5,517,10:15:00,KLKV,KRKM,VLRD,10:55:00,17.0,00:40:00,565.0
6,582,20:20:00,VLRD,KRKM,KLKV,21:00:00,17.0,00:40:00,155.0
7,123,23:35:00,KLKV,NH,PSL,23:45:00,3.5,00:10:00,NaN


Steering Duty =  07:25:00
Spread Over =  20:05:00


IndexError: single positional indexer is out-of-bounds

In [ ]:

""" 
Sl No.
Departure Time
Departure Place
Route of Operation
Arrival Place
Arrival Time act
KM
Running Time 
"""
print(trip)

[{'selected_leg':   Sl No. Departure Time Departure Place Route of Operation Arrival Place  \
0      1       03:40:00             PSL                 NH          KLKV   

  Arrival Time act   KM Running Time  Terminal Gap  
0         03:45:00  3.5     00:05:00             5  , 'temp_set':      Sl No. Departure Time Departure Place  \
0        15       04:20:00             PSL   
1         4       04:30:00             PSL   
2       693       05:00:00             PSL   
3       645       05:00:00             PSL   
4         6       05:05:00             PSL   
..      ...            ...             ...   
98       62       14:35:00             PSL   
99       63       14:40:00             PSL   
100      55       15:05:00             PSL   
101      65       15:20:00             PSL   
102     531       15:40:00             PSL   

                     Route of Operation Arrival Place Arrival Time act    KM  \
0                                    NH          KLKV         04:25:00   3.5 